In [1]:
setwd("/Users/amyvandiver/Box/Nanopore/Timp_data/HV")

In [2]:
library(GenomicRanges)
library(GenomicAlignments)
library(ggbio)
library(ggplot2)
library(gridExtra)

Warning message:
“package ‘GenomicRanges’ was built under R version 4.0.3”
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.0.3”
Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort

In [11]:
#Read in individual runs with genomic alignments 

flag0 <- scanBamFlag(isDuplicate=FALSE, isNotPassingQualityControls=FALSE)
param0 <- ScanBamParam(flag=flag0, what="seq")
HV1=readGAlignments("210617_HV_mito_targseq_2.gup5_rot.bam",use.names=TRUE,param=param0)
    HV1=HV1[which(seqnames(HV1)=="chrM_rot")]

HV2=readGAlignments("190226_HV_mitoenrich.gup5_rot.bam",use.names=TRUE,param=param0)
    HV2=HV2[which(seqnames(HV2)=="chrM_rot")]

In [12]:
getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}

In [13]:
length(HV1)
summary(qwidth(HV1))
getmode(qwidth(HV1))
length(which(qwidth(HV1)>15000))/length(HV1)

length(HV2)
summary(qwidth(HV2))
getmode(qwidth(HV2))
length(which(qwidth(HV2)>15000))/length(HV2)

[1] 1765

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    215   10417   16263   13592   16411   97460 

[1] 16416

[1] 0.6458924

[1] 458

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    183    6071   15292   11659   16420   59895 

[1] 16478

[1] 0.510917

In [144]:
#read in merged file with genomic alignments 
HV=readGAlignments("HV_all.bam",use.names=TRUE,param=param0)

#load genotype file from python, add start and ends to this from bam
geno=read.csv("geno.csv")
HV=HV[which(names(HV)%in%geno$read)]

geno$geno1=substr(as.character(geno$geno),1,1)
geno$geno2=substr(as.character(geno$geno),2,2)

idx=which(names(HV)%in%geno$read)
idx=idx[-(length(idx)-1)]
geno$start=start(HV)[idx]
geno$end=end(HV)[idx]

In [262]:
#check that matches 
summary(factor(geno$geno1))
summary(factor(geno$geno2))
summary(factor(geno$geno))

-    A    C    G    T 
  34  108    5 1297    3

-    A    C    G    T 
  34  102    1 1309    1

-A   -G   A-   AA   AG   CG   G-   GA   GC   GG   GT   TG 
   3   31    2   19   87    5   32   80    1 1183    1    3

In [149]:
#create subsample with 10% of mutants, 2.5% of wildtypes. Add y axis and colors for plotting

AA=geno[which(geno$geno=="AA"),]
AG=geno[which(geno$geno=="AG"),]
GA=geno[which(geno$geno=="GA"),]
GG=geno[which(geno$geno=="GG"),]

nano=rbind(AG[sample(nrow(AG),round(0.1*nrow(AG),0)),],
    AA[sample(nrow(AA),round(0.1*nrow(AA),0)),],
      GA[sample(nrow(GA),round(0.1*nrow(GA),0)),],
      GG[sample(nrow(GG),round(0.025*nrow(GG),0)),])
nano$y=rev(seq(40,nrow(nano)*40,by=40))

nano$col1="steelblue4"
nano$col1[which(nano$geno1=="A")]="red4"
nano$col2="steelblue4"
nano$col2[which(nano$geno2=="A")]="red4"

In [150]:
#unrotate the genome so it matches up
nano$len=nano$end-nano$start
nano$start[which(nano$start > 15022)]=nano$start[which(nano$start > 15022)]-15022
nano$start[which(nano$start < 15022)]=nano$start[which(nano$start < 15022)]+1547
nano$end=nano$start+nano$len


In [170]:
#read in illumina alignments and genotype files from pythong
HV=readGAlignments("illumina/mito_S1.bam",use.names=TRUE,param=param0)

geno1=read.csv("illumina/geno_1642.csv")
geno2=read.csv("illumina/geno_13513.csv")



In [232]:
#add coordinates to first illumina genotype file
HV1=HV[which(start(HV)<1642&end(HV)>1642)]
HV1=HV1[which(!duplicated(names(HV1)))]
geno1=geno1[which(geno1$read%in%names(HV1)),]
idx=which(names(HV1)%in%geno1$read)

geno1$start=start(HV1)[idx]
geno1$end=end(HV1)[idx]

In [234]:
#add coordinate to second illumina genotype file
HV2=HV[which(start(HV)<13513&end(HV)>13513)]
HV2=HV2[which(!duplicated(names(HV2)))]
geno2=geno2[which(geno2$read%in%names(HV2)),]

idx=which(names(HV2)%in%geno2$read)

geno2$start=start(HV2)[idx]
geno2$end=end(HV2)[idx]

In [248]:
#select random sample of illumina reads covering each snp, add y coordinates for plotting
set.seed("13579")
ymax=max(nano$y)+80
summary(factor(geno1$geno))
illum1=geno1[sample(nrow(geno1),nrow(nano)),]
illum1$y=seq(ymax,ymax-40+(nrow(illum1)*40),by=40)
illum2=geno2[sample(nrow(geno2),nrow(nano)),]
illum2$y=seq(ymax,ymax-40+(nrow(illum2)*40),by=40)



A   G 
 13 199

In [249]:
#Check distribution of random reads 
summary(factor(illum1$geno))
summary(factor(illum2$geno))

A  G 
 4 45

A  G 
 5 44

In [253]:
#color illumina sample reads by genotype
illum1$col="steelblue4"
illum1$col[which(illum1$geno=="A")]="red4"

illum1$col1="lightblue"
illum1$col1[which(illum1$geno=="A")]="rosybrown1"

illum2$col="steelblue4"
illum2$col[which(illum2$geno=="A")]="red4"

illum2$col1="lightblue"
illum2$col1[which(illum2$geno=="A")]="rosybrown1"

In [259]:
#plot both random samples together 

pdf("Illum_Nano.pdf",width=7,height=10)

plot(c(0,18000),c(min(nano$y),max(illum1$y)),type="n",ylab="",xlab="Location on ChrM",yaxt="n")

for (i in 1:(nrow(nano))){
    rect(nano$start[i],nano$y[i],nano$end[i],nano$y[i]+15,col="lightblue", border="NA")
    if(nano$geno1[i]=="A"){
        rect(1547,nano$y[i],2050,nano$y[i]+15,col="rosybrown1",border="NA")
        }
    if(nano$geno2[i]=="A"){
        rect(13200,nano$y[i],13700,nano$y[i]+15,col="rosybrown1",border="NA")
        }
    text(1642,nano$y[i]+5,labels=nano$geno1[i],col=nano$col1[i],cex=0.6)
    text(13513,nano$y[i]+5,labels=nano$geno2[i],col=nano$col2[i],cex=0.6)

}

for (i in 1:(nrow(illum1))){
    rect(illum1$start[i]-200,illum1$y[i],illum1$end[i]+200,illum1$y[i]+15,col=illum1$col1[i],lty=0)
    text(1642,illum1$y[i]+7,labels=illum1$geno[i],col=illum1$col[i],cex=0.6)
}

for (i in 1:(nrow(illum2))){
    rect(illum2$start[i]-200,illum2$y[i],illum2$end[i]+200,illum2$y[i]+15,col=illum2$col1[i],lty=0)
    text(13531,illum2$y[i]+7,labels=illum2$geno[i],col=illum2$col[i],cex=0.6)
}
text(100, max(nano$y)-500, "Nanopore Sequencing",pos = 2, srt = 90)
text(100, max(illum1$y), "Illumina Sequencing",pos = 2, srt = 90)
abline(h=max(nano$y)+40,lty=3)

dev.off()

pdf 
  2

In [95]:
install.packages("UpSetR")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [96]:
library(UpSetR)

In [104]:
#make upset plot based on numbers of AG, GA and AA from python analysis (same as python plot in prior script, easier to manipulate image)
input <- c(Snp1 = 87, Snp2 = 80, `Snp1&Snp2` = 19)


# Plot
pdf("Upset.pdf",width=5,height=7)
upset(fromExpression(input), 
#      nintersects = 40, 
#      nsets = 6, 
      order.by = "freq", 
      decreasing = T, 
      mb.ratio = c(0.6, 0.4),
      number.angles = 0, 
      text.scale = 1.5, 
      point.size = 2.8, 
      line.size = 1
      )

dev.off()

pdf 
  2